In [1]:
!pip install -q tfds-nightly

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [3]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [4]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:29<00:00, 29.52s/ url]


 28%|██▊       | 14075/50000 [00:00<00:00, 62033.20 examples/s]Shuffling and writing examples to C:\Users\jedgo\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompleteC80QWI\imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to C:\Users\jedgo\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0. Subsequent calls will reuse this data.


In [7]:
encoder = info.features['text'].encoder

In [8]:
print('Vocabulary size {}'.format(encoder.vocab_size))

Vocabulary size 8185


In [9]:
sample_string = 'Hello Tensorflow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}""'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 2934, 7975]
The original string: "Hello Tensorflow.""


In [10]:
assert original_string == sample_string

In [11]:
for index in encoded_string:
    print('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
2934 ----> flow
7975 ----> .


In [12]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [13]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_dataset.padded_batch(BATCH_SIZE)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [15]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [17]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

Epoch 1/10
 13/391 [..............................] - ETA: 1:14 - loss: 0.6930 - accuracy: 0.5024

CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node gradient_tape/sequential/embedding/embedding_lookup/Reshape/_38}}]] [Op:__inference_train_function_5998]

Function call stack:
train_function
